In [ ]:
lgb_params = {
                    'objective':'binary',
                    'boosting_type':'gbdt',
                    'metric':'auc',
                    'n_jobs':-1,
                    'learning_rate':0.01,
                    'num_leaves': 2**8,
                    'max_depth':-1,
                    'tree_learner':'serial',
                    'colsample_bytree': 0.7,
                    'subsample_freq':1,
                    'subsample':0.7,
                    'n_estimators':80000,
                    'max_bin':255,
                    'verbose':-1,
                    'seed':42,
                    'early_stopping_rounds':100, 
                } 

In [ ]:
fq_encode = []
base_columns = list(full_df)

In [1]:
#总之思想就是每次训练完记录一下现在的特征状态，然后记录一下AUC和上一次的比较
def make_test(old_score=0, output=False):
    features_columns = [col for col in list(full_df) if col not in remove_features]
    train_mask = full_df['TransactionID'].isin(local_train_id['TransactionID'])#这个地方放入train 和test的 id
    test_mask = full_df['TransactionID'].isin(local_test_id['TransactionID'])
    
    X,y = full_df[train_mask][features_columns], full_df[train_mask][TARGET]    
    P,P_y = full_df[test_mask][features_columns], full_df[test_mask][TARGET]  

    for col in list(X):
        if X[col].dtype=='O':
            X[col] = X[col].fillna('unseen_before_label')
            P[col] = P[col].fillna('unseen_before_label')

            X[col] = X[col].astype(str)
            P[col] = P[col].astype(str)

            le = LabelEncoder()
            le.fit(list(X[col])+list(P[col]))
            X[col] = le.transform(X[col])
            P[col]  = le.transform(P[col])

            X[col] = X[col].astype('category')
            P[col] = P[col].astype('category')
        
    tt_df = full_df[test_mask][['TransactionID','DT_W',TARGET]]        
    tt_df['prediction'] = 0
    
    tr_data = lgb.Dataset(X, label=y)
    vl_data = lgb.Dataset(P, label=P_y) 
    estimator = lgb.train(
            lgb_params,
            tr_data,
            valid_sets = [tr_data, vl_data],
            verbose_eval = 200,
        )   
        
    tt_df['prediction'] = estimator.predict(P)
    feature_imp = pd.DataFrame(sorted(zip(estimator.feature_importance(),X.columns)), columns=['Value','Feature'])
    
    if output:
        tt_df[['TransactionID','prediction']].to_csv('oof.csv',index=False)
        print('---Wrote OOF to file---')
    
    m_results = []
    print('#'*20)
    g_auc = metrics.roc_auc_score(tt_df[TARGET], tt_df['prediction'])
    score_diff = g_auc - old_score
    print('Global AUC', g_auc)
    m_results.append(g_auc)
    
    for i in range(full_df[test_mask]['DT_W'].min(), full_df[test_mask]['DT_W'].max()+1):
        mask = tt_df['DT_W']==i
        w_auc = metrics.roc_auc_score(tt_df[mask][TARGET], tt_df[mask]['prediction'])
        print('Week', i, w_auc, len(tt_df[mask]))
        m_results.append(w_auc)
        
    print('#'*20)
    print('Features Preformance:', g_auc)
    print('Diff with previous__:', score_diff)
    
    return tt_df, feature_imp, m_results, estimator

In [ ]:
def get_new_columns(temp_list):
    temp_list = [col for col in list(full_df) if col not in temp_list]
    temp_list.sort()

    temp_list2 = [col if col not in remove_features else '-' for col in temp_list ]
    temp_list2.sort()

    temp_list = {'New columns (including dummy)': temp_list,
                 'New Features': temp_list2}
    temp_list = pd.DataFrame.from_dict(temp_list)
    return temp_list

In [ ]:
#baseline
tt_df, feature_imp, m_results, model = make_test()

In [ ]:
saved_state = list(full_df)
#每次先保存状态，再进行下一轮特征工程

In [ ]:
print(get_new_columns(saved_state))
tt_df, feature_imp, m_results, model = make_test(m_results[0])